In [36]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
print(tf.__version__)

2.16.1


In [37]:
df_train = pd.read_csv(r'C:\Users\USER\Desktop\sales pred\Train.csv')
df_test = pd.read_csv(r'C:\Users\USER\Desktop\sales pred\Test.csv')

In [38]:
def evaluate_model(true_values, predicted_values):
    rmse = np.sqrt(mean_squared_error(true_values, predicted_values))
    mae = mean_absolute_error(true_values, predicted_values)
    r2 = r2_score(true_values, predicted_values)
    return mae, rmse, r2

In [42]:
# Data preprocessing (fixed typo: Outlet_Location -> Outlet_Location_Type)
categorical_cols = ['Item_Fat_Content', 'Item_Type', 'Outlet_Location', 'Outlet_Type']
numerical_cols = ['Item_Weight', 'Item_MRP', 'Outlet_Establishment_Year']

# Check NaNs
print("NaN counts in df_train:")
print(df_train.isna().sum())

# Handle NaNs
df_train[numerical_cols] = df_train[numerical_cols].fillna(df_train[numerical_cols].median())
df_train = df_train.dropna(subset=['Item_Outlet_Sales'])

NaN counts in df_train:
Item_Weight                  0
Item_Fat_Content             0
Item_Type                    0
Item_MRP                     0
Outlet_Establishment_Year    0
Outlet_Location              0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64


In [43]:
# Encode categorical variables
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df_train[col] = le.fit_transform(df_train[col])
    label_encoders[col] = le

In [44]:
# Split features and target
X = df_train.drop('Item_Outlet_Sales', axis=1)
y = df_train['Item_Outlet_Sales']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the target (new!)
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1)).flatten()

# Debug
print("Sample y_train values:", y_train[:5].values)

Sample y_train values: [ 8394140   156169  1422611 19285740 11673761]


In [45]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [46]:
# Define a deeper TensorFlow model with dropout
n_features = X_train_scaled.shape[1]
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(n_features,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),  # Prevent overfitting
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer
])

# Compile with adjusted learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), 
              loss='mean_squared_error')

In [47]:
# Train the model on scaled target
history = model.fit(X_train_scaled, y_train_scaled, 
                    validation_data=(X_test_scaled, y_test_scaled),
                    epochs=300,  # More epochs for better convergence
                    batch_size=32, 
                    verbose=1)

Epoch 1/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 149ms/step - loss: 1.0744 - val_loss: 1.1913
Epoch 2/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 1.0097 - val_loss: 1.1358
Epoch 3/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.8573 - val_loss: 1.0868
Epoch 4/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.8140 - val_loss: 1.0414
Epoch 5/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.7828 - val_loss: 0.9920
Epoch 6/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - loss: 0.7977 - val_loss: 0.9486
Epoch 7/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - loss: 0.7626 - val_loss: 0.9146
Epoch 8/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.6934 - val_loss: 0.8870
Epoch 9/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - loss: 0.6532 - val_loss: 0.8625
Epoch 10/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - loss: 0.6892 - val_loss: 0.8440
Epoch 11/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.6751 - val_loss: 0.8212
Epoch 12/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 0.5758 - val_l

In [48]:
# Predictions (in scaled space)
y_train_pred_scaled = model.predict(X_train_scaled).flatten()
y_test_pred_scaled = model.predict(X_test_scaled).flatten()

# Inverse transform to UGX
y_train_pred = y_scaler.inverse_transform(y_train_pred_scaled.reshape(-1, 1)).flatten()
y_test_pred = y_scaler.inverse_transform(y_test_pred_scaled.reshape(-1, 1)).flatten()

# Debug
print("Sample y_train_pred:", y_train_pred[:5])
print("Sample y_test_pred:", y_test_pred[:5])

# Evaluate
train_mae, train_rmse, train_r2 = evaluate_model(y_train, y_train_pred)
test_mae, test_rmse, test_r2 = evaluate_model(y_test, y_test_pred)

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Sample y_train_pred: [ 5661259.   2042660.1  1823629.1 11394803.  13260964. ]
Sample y_test_pred: [11826518. 10439182.  5215487.  2612972.  7912818.]


In [49]:
# Print results
print('TensorFlow Improved Neural Network')
print('Model performance for Training set')
print(f'- Root Mean Squared Error: {train_rmse:.4f} UGX')
print(f'- Mean Absolute Error: {train_mae:.4f} UGX')
print(f'- R2 Score: {train_r2:.4f}')
print('----------------------------------')
print('Model performance for Test set')
print(f'- Root Mean Squared Error: {test_rmse:.4f} UGX')
print(f'- Mean Absolute Error: {test_mae:.4f} UGX')
print(f'- R2 Score: {test_r2:.4f}')
print('='*35)

# Save results
file_path = r'C:\Users\USER\Desktop\sales pred\notebook\tensorflow_improved_results.txt'
with open(file_path, 'w') as f:
    f.write('TensorFlow Improved Neural Network\n')
    f.write('Model performance for Training set\n')
    f.write(f'- Root Mean Squared Error: {train_rmse:.4f} UGX\n')
    f.write(f'- Mean Absolute Error: {train_mae:.4f} UGX\n')
    f.write(f'- R2 Score: {train_r2:.4f}\n')
    f.write('----------------------------------\n')
    f.write('Model performance for Test set\n')
    f.write(f'- Root Mean Squared Error: {test_rmse:.4f} UGX\n')
    f.write(f'- Mean Absolute Error: {test_mae:.4f} UGX\n')
    f.write(f'- R2 Score: {test_r2:.4f}\n')
    f.write('='*35 + '\n')
print(f'Results saved to {file_path}')

TensorFlow Improved Neural Network
Model performance for Training set
- Root Mean Squared Error: 3598808.0239 UGX
- Mean Absolute Error: 2738592.7323 UGX
- R2 Score: 0.6692
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5805413.5499 UGX
- Mean Absolute Error: 4401111.0938 UGX
- R2 Score: 0.2881
Results saved to C:\Users\USER\Desktop\sales pred\notebook\tensorflow_improved_results.txt
